In [1]:
import os
import nbformat #for running jupyter notebooks
import nbparameterise as nbparam #for changing input parameters in jupyter notebooks

# Measure the Future - Visualizations
#### Overview
Measure the Future involves placement of fixed cameras, called scouts, that track movements through public places.  We want to visualize the data collected by scouts in order to draw meaningful conclusions from it about where people tend to go and when they go there.

#### Format MtF data should be in before using this notebook
Each scout should have a folder containing data recorded by that scout.  The name of the folder should be the name of that scout, for example "Main 1 - LobbyHall".

A scout's folder should have several subfolders containing data from different ranges of dates.  The names of these folders should be of the form:
* [shotened scout name]\_[end date of range in the form MMDDYYYY]

Each of these subfolders should contain a jpeg calibration image, and four json files: scout_interactions, scout_summaries, scouts, and scout_healths. 

#### Calibration image
Filename should be of the form:
* scout-[a long string on letters and numbers].jpg

This shows the view the scout sees during the date range of the subfolder it's in.  The reason there is a separete calibration image for each subfolder is that getting the data out of the scout and deleting it in preparation from more data collection involves touching the scout.  Doing so may move the scout slightly, so it must be recalibrated after.  Thus, the calibration images will not always be identical for different date ranges.

#### scout_interactions.json
Each "interaction" represents one person's path across the scout's frame of view.  An interaction is represented by a list of waypoints, represented by 2d coordinates on the calibration image.  

In addition, interactions are recorded with a duration (in seconds), an enter time (in UTC), a series of waypoint times (in seconds after the enter time), and a series of waypoint widths (representing the recorded width and height of the person at each waypoint).

#### Visualiztion Notebooks
There should be a set of notebooks (stored in the same folder as this one), each of which does one particular visualization on one particular scout.  You can choose which of these notebooks to run, and each one you choose will be run on every scout.

Some notebooks only do visualizations on the most recent date range subfolder for a scout.  Others combine the data across all date ranges.

#### Constants
Make sure these are correct before you run this notebook!

In [2]:
#Set the folder containing all the MtF data you want to visualize:
#(It is assumed that all subfolders of this folder are data for one scout)
MTF_PATH = None
os.listdir(MTF_PATH) #take a look at which scouts we'll be visualizing

['DB1 - Entry',
 'DB2 - Childrens',
 'Main 1 - LobbyHall',
 'Main 2 - LobbyCS',
 'Main 3 - MFA']

In [3]:
scout_folder_paths = []
for name in  os.listdir(MTF_PATH):
    if os.path.isdir(MTF_PATH + os.sep + name):
        scout_folder_paths.append(MTF_PATH + os.sep + name)

In [4]:
#take a look at the aviailable visualization notebooks
this_directory = os.listdir()
this_directory

['.ipynb_checkpoints',
 'load_mtf_data.py',
 'main.ipynb',
 'misc not used code',
 'mtf_data',
 'mtf_data_old (from eleanor thumb drive)',
 'mtf_data_old2 (before switched to onedrive format_',
 'mtf_visualize_1.ipynb',
 'mtf_visualize_2_flexible.ipynb',
 'mtf_visualize_3_more_experiments.ipynb',
 'mtf_visualize_4_uses_path.ipynb',
 'mtf_visualize_4_uses_path_executed',
 'mtf_visualize_5_heatmap.ipynb',
 'mtf_visualize_5_paths.ipynb',
 'mtf_visualize_5_paths_executed',
 'mtf_visualize_5_remove_outliers.ipynb',
 'mtf_visualize_5_time.ipynb',
 'mtf_visualize_5_time_executed',
 'mtf_visualize_6_time.ipynb',
 'mtf_visualize_7_time.ipynb',
 'mtf_visualize_7_time_executed',
 '__pycache__']

In [5]:
#set the names of mtf_visualize notebooks to use
MTF_VISUALIZE_NAMES = ['mtf_visualize_7_time', 'mtf_visualize_5_heatmap', 'mtf_visualize_5_paths']
for name in MTF_VISUALIZE_NAMES: assert(name + ".ipynb" in this_directory)

For each selected mtf_visualize notbook, run that notebook on all scouts.

Visualizations will be saved as images in the scout's folder. 

Folders will be created (in the same location as main.ipynb) containing executed versions of each notebook.  Thier names will be of the form:
* [notebook name]\_executed

In [6]:
for notebook_name in MTF_VISUALIZE_NAMES:
    
    #make directory for executed verison
    mtf_visualize_executed_dir = notebook_name + "_executed"
    try:
        os.mkdir(mtf_visualize_executed_dir)
    except FileExistsError:
        pass #it already exists, do nothing
    
    for scout_folder_path in scout_folder_paths:
        
        #use nbformat to read mtf_visualize file into a notebook object
        with open(notebook_name + ".ipynb") as f:
            nb = nbformat.read(f, as_version=4)
    
        #replace SCOUT_FOLDER_PATH parameter in first cell of notebook we want to run
        orig_params = nbparam.extract_parameters(nb)
        new_params = nbparam.parameter_values(orig_params, SCOUT_FOLDER_PATH = scout_folder_path)
        new_nb = nbparam.replace_definitions(nb, new_params)
        #new notebook gets executed automatically
    
        #save the new notebook, executed with replaced params.
        executed_notebook_name = notebook_name + "_" + os.path.basename(scout_folder_path) + ".ipynb"
        with open(mtf_visualize_executed_dir + os.sep + executed_notebook_name, 'w') as f:
            nbformat.write(new_nb, f)
 